# 🌙 EALLIS: Enhanced Adaptive Low-Light Instance Segmentation — Training Notebook

This notebook trains the **Mask R-CNN + AWD + SCB + DSL** model.

**Training**: COCO train2017 with synthetic noise pipeline (SynCOCO)  
**Evaluation**: EALLIS test set (real-world dark images)  
**Model**: `MaskRCNNNoiseInv` with `ResNetAdaDSmoothPrior` backbone

---
⚠️ **Requirements**: GPU runtime (T4 or better). Go to `Runtime > Change runtime type > GPU`.

## 1. Mount Google Drive & Check GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import torch, sys
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.version.cuda}')
print(f'Python: {sys.version}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 2. Install Dependencies

This cell caches the compiled mmcv-full on Google Drive. First run takes ~20-30 min, but subsequent sessions install in **~30 seconds** from the cached wheel.

In [ ]:
import os, glob, time
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda.replace('.', '')
py_ver = f'{sys.version_info.major}{sys.version_info.minor}'

# Cache directory on Google Drive
CACHE_DIR = '/content/drive/MyDrive/EALLIS_pip_cache'
os.makedirs(CACHE_DIR, exist_ok=True)

# Check if mmcv is already installed
try:
    import mmcv
    print(f'✅ mmcv-full {mmcv.__version__} already installed! Skipping.')
except ImportError:
    # Check for cached wheel on Drive
    cached_wheels = glob.glob(os.path.join(CACHE_DIR, 'mmcv_full-1.7.2*.whl'))
    
    if cached_wheels:
        print(f'📦 Found cached wheel on Drive, installing (~30 sec)...')
        wheel_path = cached_wheels[0]
        os.system(f'pip install {wheel_path}')
    else:
        print(f'⏳ No cached wheel found. Compiling mmcv-full from source (~20-30 min)...')
        print(f'   PyTorch {torch_ver}, CUDA {cuda_ver}, Python {py_ver}')
        print('='*60)
        
        # Build a wheel so we can cache it
        os.system(f'pip wheel mmcv-full==1.7.2 -w /tmp/mmcv_wheels '
                  f'-f https://download.openmmlab.com/mmcv/dist/cu{cuda_ver}/torch{torch_ver}/index.html '
                  f'2>&1 | tail -5')
        
        # Install the built wheel
        built_wheels = glob.glob('/tmp/mmcv_wheels/mmcv_full-1.7.2*.whl')
        if built_wheels:
            wheel_path = built_wheels[0]
            os.system(f'pip install {wheel_path}')
            # Cache to Drive for next time
            import shutil
            shutil.copy2(wheel_path, CACHE_DIR)
            print(f'\n💾 Cached wheel to Drive for future sessions!')
        else:
            # Fallback: direct pip install
            os.system(f'pip install mmcv-full==1.7.2 '
                      f'-f https://download.openmmlab.com/mmcv/dist/cu{cuda_ver}/torch{torch_ver}/index.html')
    
    import mmcv
    print(f'\n✅ mmcv-full {mmcv.__version__} installed!')

# Other dependencies (fast)
!pip install -q pycocotools scikit-learn terminaltables pretrainedmodels
print('✅ All dependencies ready!')

## 3. Clone the EALLIS Repository

In [ ]:
import os

REPO_DIR = '/content/EALLIS'

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/itzaqeel/EALLIS.git {REPO_DIR}
else:
    print(f'{REPO_DIR} already exists, skipping clone.')

os.chdir(REPO_DIR)
print(f'Working directory: {os.getcwd()}')

## 4. Install mmdetection (from local repo)

In [ ]:
import sys
os.chdir(os.path.join(REPO_DIR, 'mmdetection'))
!pip install -e . --no-deps
os.chdir(REPO_DIR)

sys.path.insert(0, os.path.join(REPO_DIR, 'mmdetection'))

import mmdet
print(f'mmdet version: {mmdet.__version__}')

## 5. Apply All Fixes

Fixes compatibility issues with Python 3.12, mmcv 1.7.2, and the custom modules.

In [ ]:
import glob
import re

os.chdir(REPO_DIR)

# Fix 1: Bump mmcv version cap
init_file = 'mmdetection/mmdet/__init__.py'
with open(init_file, 'r') as f:
    content = f.read()
content = re.sub(r"mmcv_maximum_version\s*=\s*'[^']*'", "mmcv_maximum_version = '3.0.0'", content)
with open(init_file, 'w') as f:
    f.write(content)
print('[Fix 1] mmcv version cap updated.')

# Fix 2: Comment out deprecated/removed imports (Python 3.12 compatibility)
for resnet_file in [
    'mmdetection/mmdet/models/backbones/resnet.py',
    'mmdetection_custom_part/mmdet/models/backbones/resnet.py'
]:
    if os.path.exists(resnet_file):
        with open(resnet_file, 'r') as f:
            content = f.read()
        content = content.replace('import imp', '# import imp  # removed in Python 3.12')
        content = content.replace('from os import pread', '# from os import pread')
        content = content.replace('from tokenize import group', '# from tokenize import group')
        content = content.replace('from numpy.core.fromnumeric import size', '# from numpy.core.fromnumeric import size')
        content = content.replace('from torch.functional import _index_tensor_with_indices_list',
                                  '# from torch.functional import _index_tensor_with_indices_list')
        with open(resnet_file, 'w') as f:
            f.write(content)
print('[Fix 2] Python 3.12 deprecated imports fixed.')

# Fix 3: Fix relative imports
import_fixes = [
    ('mmdetection_custom_part/mmdet/models/backbones/resnet.py', 'from ..utils', 'from mmdet.models.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/resnet.py', 'from .cbam', 'from mmdet.models.backbones.cbam'),
    ('mmdetection_custom_part/mmdet/models/backbones/resnext.py', 'from ..utils', 'from mmdet.models.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py', 'from ...utils', 'from mmdet.utils'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py', 'from ..utils.ckpt_convert', 'from mmdet.models.utils.ckpt_convert'),
    ('mmdetection_custom_part/mmdet/models/backbones/swin.py', 'from ..utils.transformer', 'from mmdet.models.utils.transformer'),
    ('mmdetection_custom_part/mmdet/models/dense_heads/maskformer_head.py', 'from .anchor_free_head', 'from mmdet.models.dense_heads.anchor_free_head'),
    ('mmdetection_custom_part/mmdet/models/dense_heads/mask2former_head.py', 'from .anchor_free_head', 'from mmdet.models.dense_heads.anchor_free_head'),
    ('mmdetection_custom_part/mmdet/models/detectors/two_stage.py', 'from .base', 'from mmdet.models.detectors.base'),
    ('mmdetection_custom_part/mmdet/models/detectors/maskformer.py', 'from .single_stage', 'from mmdet.models.detectors.single_stage'),
    ('mmdetection_custom_part/mmdet/models/seg_heads/base_semantic_head.py', 'from ..utils import interpolate_as', 'from mmdet.models.utils import interpolate_as'),
    ('mmdetection_custom_part/mmdet/models/seg_heads/panoptic_fpn_head.py', 'from ..utils import ConvUpsample', 'from mmdet.models.utils import ConvUpsample'),
]
for filepath, old_imp, new_imp in import_fixes:
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            content = f.read()
        if old_imp in content:
            content = content.replace(old_imp, new_imp)
            with open(filepath, 'w') as f:
                f.write(content)
print('[Fix 3] Relative imports fixed.')

# Fix 4: Detector auxiliary imports
for det_file in [
    'mmdetection_custom_part/mmdet/models/detectors/mask_rcnn.py',
    'mmdetection_custom_part/mmdet/models/detectors/faster_rcnn_noise_inv.py'
]:
    if os.path.exists(det_file):
        with open(det_file, 'r') as f:
            content = f.read()
        content = content.replace('from ..backbones.aux_modules', 'from mmdet.models.backbones.aux_modules')
        content = content.replace('from ..backbones.multiscale_discriminator', 'from mmdet.models.backbones.multiscale_discriminator')
        content = content.replace('from ..backbones.lsid', 'from mmdet.models.backbones.lsid')
        with open(det_file, 'w') as f:
            f.write(content)
print('[Fix 4] Detector auxiliary imports fixed.')

# Fix 5: Force register_module()
custom_dir = 'mmdetection_custom_part'
count = 0
for py_file in glob.glob(os.path.join(custom_dir, '**', '*.py'), recursive=True):
    with open(py_file, 'r') as f:
        content = f.read()
    if '.register_module()' in content:
        new_content = content.replace('.register_module()', '.register_module(force=True)')
        with open(py_file, 'w') as f:
            f.write(new_content)
        count += 1
print(f'[Fix 5] Forced registration in {count} files.')

# Fix 6: Rewrite __init__.py files
with open('mmdetection_custom_part/mmdet/models/backbones/__init__.py', 'w') as f:
    f.write("""from .resnet import ResNet, ResNetV1d, ResNetAdaD, ResNetAdaDSmoothPrior
from .resnext import ResNeXt
from .swin import SwinTransformer, SwinTransformerV2
from .convnext import ConvNeXt
__all__ = ['ResNet', 'ResNetV1d', 'ResNeXt', 'SwinTransformer', 'SwinTransformerV2',
           'ConvNeXt', 'ResNetAdaD', 'ResNetAdaDSmoothPrior']
""")
with open('mmdetection_custom_part/mmdet/models/dense_heads/__init__.py', 'w') as f:
    f.write("""from .maskformer_head import MaskFormerHead
from .mask2former_head import Mask2FormerHead
__all__ = ['MaskFormerHead', 'Mask2FormerHead']
""")
with open('mmdetection_custom_part/mmdet/models/detectors/__init__.py', 'w') as f:
    f.write("""from .two_stage import TwoStageDetector
from .faster_rcnn import FasterRCNN
from .faster_rcnn_noise_inv import FasterRCNNNoiseInv
from .mask_rcnn import MaskRCNN, MaskRCNNNoiseInv as MaskRCNNNoiseInvDet
from .maskformer import MaskFormer
from .mask2former import Mask2Former
__all__ = ['TwoStageDetector', 'FasterRCNN', 'FasterRCNNNoiseInv',
           'MaskRCNN', 'MaskRCNNNoiseInvDet', 'MaskFormer', 'Mask2Former']
""")
models_init = 'mmdetection_custom_part/mmdet/models/__init__.py'
if os.path.exists(models_init):
    with open(models_init, 'r') as f:
        content = f.read()
    content = content.replace('from .necks import *', '# from .necks import *')
    with open(models_init, 'w') as f:
        f.write(content)
print('[Fix 6] __init__.py files rewritten.')
print('\n✅ All fixes applied!')

## 6. Verify Custom Modules

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)
sys.path.insert(0, os.path.join(REPO_DIR, 'mmdetection'))

try:
    import mmdetection_custom_part.mmdet.models.detectors
    import mmdetection_custom_part.mmdet.models.backbones
    import mmdetection_custom_part.mmdet.models.dense_heads
    import mmdetection_custom_part.mmdet.models.roi_heads
    import mmdetection_custom_part.mmdet.models.plugins
    import mmdetection_custom_part.mmdet.models.seg_heads
    import mmdetection_custom_part.mmdet.models.losses
    print('✅ All custom modules imported successfully!')
except Exception as e:
    import traceback
    traceback.print_exc()
    raise RuntimeError(f'Custom module import failed: {e}')

## 7. Setup Datasets

In [ ]:
os.chdir(REPO_DIR)
os.makedirs('data/coco/annotations', exist_ok=True)
os.makedirs('data/coco/train2017', exist_ok=True)
os.makedirs('data/eallis/annotations', exist_ok=True)
os.makedirs('data/eallis/images', exist_ok=True)
print('Data directories created.')

### 7a. Download COCO

In [ ]:
DOWNLOAD_COCO = True  # Set False if using Drive symlinks

if DOWNLOAD_COCO:
    os.chdir(os.path.join(REPO_DIR, 'data/coco'))
    !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -qo annotations_trainval2017.zip
    !rm annotations_trainval2017.zip
    !wget -q http://images.cocodataset.org/zips/train2017.zip
    !unzip -qo train2017.zip
    !rm train2017.zip
    os.chdir(REPO_DIR)
    print(f'COCO train images: {len(os.listdir("data/coco/train2017"))}')
else:
    print('Skipping COCO download.')
    # Uncomment to symlink from Drive:
    # !ln -sf /content/drive/MyDrive/datasets/coco/train2017 data/coco/train2017
    # !ln -sf /content/drive/MyDrive/datasets/coco/annotations data/coco/annotations

### 7b. Create 10% COCO Subset (Optional)

Set `USE_SUBSET = False` for full COCO training.

In [ ]:
import json
import random

os.chdir(REPO_DIR)

USE_SUBSET = True       # Set False to train on full COCO
SUBSET_RATIO = 0.10     # 10% of COCO (~11.8K images)

if USE_SUBSET:
    ann_path = 'data/coco/annotations/instances_train2017.json'
    subset_path = 'data/coco/annotations/instances_train2017_subset.json'

    print(f'Loading full COCO annotations...')
    with open(ann_path, 'r') as f:
        coco_data = json.load(f)

    all_images = coco_data['images']
    num_subset = int(len(all_images) * SUBSET_RATIO)

    random.seed(42)
    subset_images = random.sample(all_images, num_subset)
    subset_img_ids = set(img['id'] for img in subset_images)
    subset_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in subset_img_ids]

    subset_data = {
        'info': coco_data.get('info', {}),
        'licenses': coco_data.get('licenses', []),
        'images': subset_images,
        'annotations': subset_annotations,
        'categories': coco_data['categories']
    }

    with open(subset_path, 'w') as f:
        json.dump(subset_data, f)

    print(f'✅ Created {SUBSET_RATIO*100:.0f}% subset: {len(subset_images):,} images, {len(subset_annotations):,} annotations')
else:
    print('Using full COCO dataset.')

### 7c. Upload EALLIS Dataset

In [ ]:
EALLIS_DRIVE_PATH = '/content/drive/MyDrive/datasets/eallis'

if os.path.exists(EALLIS_DRIVE_PATH):
    if os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'annotations')):
        !cp -r {EALLIS_DRIVE_PATH}/annotations/* {REPO_DIR}/data/eallis/annotations/
    if os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'images')):
        !cp -r {EALLIS_DRIVE_PATH}/images/* {REPO_DIR}/data/eallis/images/
    elif os.path.exists(os.path.join(EALLIS_DRIVE_PATH, 'JPEGImages')):
        !cp -r {EALLIS_DRIVE_PATH}/JPEGImages/* {REPO_DIR}/data/eallis/images/
    print(f'EALLIS annotations: {os.listdir(REPO_DIR + "/data/eallis/annotations")}')
    print(f'EALLIS images: {len(os.listdir(REPO_DIR + "/data/eallis/images"))} files')
else:
    print(f'EALLIS not found at {EALLIS_DRIVE_PATH}. Update the path above.')

In [ ]:
os.chdir(REPO_DIR)
jpeg_link = 'data/eallis/JPEGImages'
if not os.path.exists(jpeg_link):
    os.symlink(os.path.abspath('data/eallis/images'), jpeg_link)
    print(f'Created symlink: {jpeg_link} -> data/eallis/images')
print(f'EALLIS images accessible: {os.path.exists("data/eallis/JPEGImages")}')

## 8. Prepare Training Config

In [ ]:
os.chdir(REPO_DIR)

CONFIG_FILE = 'Configs/mask_rcnn_r50_fpn_caffe_AWD_SCB_DSL_SynCOCO2EALLIS.py'

with open(CONFIG_FILE, 'r') as f:
    config_content = f.read()

config_content = config_content.replace('BATCHSIZE = 8', 'BATCHSIZE = 4')

config_content = re.sub(
    r"load_from\s*=\s*'[^']*'",
    "load_from = None",
    config_content)

if USE_SUBSET:
    config_content = config_content.replace(
        "ann_file='data/coco/annotations/instances_train2017.json'",
        "ann_file='data/coco/annotations/instances_train2017_subset.json'")

DRIVE_SAVE_DIR = '/content/drive/MyDrive/EALLIS_checkpoints'

TRAIN_CONFIG = 'Configs/train_colab.py'
with open(TRAIN_CONFIG, 'w') as f:
    f.write(config_content)

subset_label = f'{SUBSET_RATIO*100:.0f}% subset' if USE_SUBSET else 'full'
print(f'Config saved: {TRAIN_CONFIG}')
print(f'  Train: COCO ({subset_label}) → SynCOCO | Val/Test: EALLIS | Batch: 4 | Epochs: 12')
print(f'  Auto-save to: {DRIVE_SAVE_DIR}')

## 9. Start Training 🚀

Checkpoints are **automatically saved to Google Drive** after every epoch.

In [ ]:
os.chdir(REPO_DIR)

import sys
sys.path.insert(0, REPO_DIR)
sys.path.insert(0, os.path.join(REPO_DIR, 'mmdetection'))

import mmdetection_custom_part.mmdet.models.detectors
import mmdetection_custom_part.mmdet.models.backbones
import mmdetection_custom_part.mmdet.models.dense_heads
import mmdetection_custom_part.mmdet.models.roi_heads
import mmdetection_custom_part.mmdet.models.plugins
import mmdetection_custom_part.mmdet.models.seg_heads
import mmdetection_custom_part.mmdet.models.losses

import shutil
import torch
from mmcv import Config
from mmcv.runner import HOOKS, Hook
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

@HOOKS.register_module(force=True)
class DriveBackupHook(Hook):
    """Copies checkpoint files to Google Drive after each epoch."""
    def __init__(self, drive_dir):
        self.drive_dir = drive_dir
        os.makedirs(drive_dir, exist_ok=True)

    def after_train_epoch(self, runner):
        epoch = runner.epoch + 1
        ckpt_name = f'epoch_{epoch}.pth'
        src = os.path.join(runner.work_dir, ckpt_name)
        if os.path.exists(src):
            dst = os.path.join(self.drive_dir, ckpt_name)
            shutil.copy2(src, dst)
            runner.logger.info(f'💾 Saved {ckpt_name} to Google Drive')
        for log_file in glob.glob(os.path.join(runner.work_dir, '*.log')):
            shutil.copy2(log_file, os.path.join(self.drive_dir, os.path.basename(log_file)))

cfg = Config.fromfile('Configs/train_colab.py')

cfg.custom_hooks = cfg.get('custom_hooks', [])
cfg.custom_hooks.append(dict(type='DriveBackupHook', drive_dir=DRIVE_SAVE_DIR, priority='LOW'))

print('Building datasets...')
datasets = [build_dataset(cfg.data.train)]
print(f'Train dataset: {len(datasets[0])} images')

print('Building model...')
model = build_detector(cfg.model)
model.init_weights()
model.CLASSES = datasets[0].CLASSES

print(f'Model: {cfg.model.type} | Backbone: {cfg.model.backbone.type}')
print(f'Training for {cfg.runner.max_epochs} epochs | Auto-saving to: {DRIVE_SAVE_DIR}')

train_detector(model, datasets, cfg, distributed=False, validate=True, meta=dict())

print('\n✅ Training complete!')

## 10. Evaluate on EALLIS Test Set

In [ ]:
os.chdir(REPO_DIR)

from mmcv.parallel import MMDataParallel
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test

import glob
ckpt_files = sorted(glob.glob(os.path.join(cfg.work_dir, 'epoch_*.pth')))
latest_ckpt = ckpt_files[-1] if ckpt_files else os.path.join(cfg.work_dir, 'latest.pth')
print(f'Using checkpoint: {latest_ckpt}')

test_dataset = build_dataset(cfg.data.test)
test_loader = build_dataloader(test_dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

cfg.model.pretrained = None
cfg.model.backbone.init_cfg = None
eval_model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(eval_model, latest_ckpt, map_location='cpu')
eval_model.CLASSES = test_dataset.CLASSES
eval_model = MMDataParallel(eval_model, device_ids=[0])
eval_model.eval()

print(f'Running inference on {len(test_dataset)} images...')
results = single_gpu_test(eval_model, test_loader, show=False)

eval_results = test_dataset.evaluate(results, metric=['bbox', 'segm'])
print('\n' + '='*60)
print('EVALUATION RESULTS')
print('='*60)
for key, val in eval_results.items():
    print(f'  {key}: {val:.4f}' if isinstance(val, float) else f'  {key}: {val}')
print('='*60)

---
## Tips

- **OOM errors**: Reduce `BATCHSIZE` in cell 8 (try 2 or 1)
- **Faster training**: Use A100 via `Runtime > Change runtime type`
- **Full COCO**: Set `USE_SUBSET = False` in cell 7b
- **Resume training**: Set `resume_from = 'work_dir/epoch_X.pth'` in config
- **Fine-tune from pretrained**: Set `load_from = 'Checkpoints/Checkpoint1.pth'`
- Checkpoints are auto-saved to Google Drive after every epoch — no data loss on Colab timeout!